In [ ]:
%matplotlib
import os
import gzip
import datetime

import numpy as np
import matplotlib.pyplot as plt

import clawpack.clawutil as clawutil
import clawpack.geoclaw.topotools as topotools
from clawpack.geoclaw.surge.storm import Storm

# Plotting of Storm Size and Track

These examples illustrate how to plot the track of the synthetic storm included in this directory and an example with Hurricane Ike.  These examples should be useful for deteremining distances that should prevent boundary condition problems with storm surge where the storm may get too close to the boundary.

## Example 1:  Read in synthetic storm data
Read in the synthetic storm `synthetic.storm` created in the `setrun.py` file and plot it.  This example includes coloring of the track to indicate the category (intensity) of the storm.  We also want this to not be in lat-long coordinates so that is also specified.

In [ ]:
storm = Storm(path="synthetic.storm", file_format="geoclaw")
fig, ax = plt.subplots()
storm.plot(ax, coordinate_system=1, track_style={})
ax.set_aspect('equal')
ax.set_title("Storm Track - Example 1")
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_xlim((-500e3, 500e3))
ax.set_ylim((-500e3, 500e3))

## Example 2:  Reading output track data
This example reads in the track from the file `fort.track` that is produced while simulating the storm.  This constitutes simply loading a `Storm` object with the data loaded from the file.  This does not set the `storm_radius` of the storm so we explicitly draw a radius of 300 km.  This example also shows how to restrict the time range via the `storm.plot` function.

In [ ]:
# Load track data
data = np.loadtxt(os.path.join(os.getcwd(), "_output", "fort.track"))
storm = Storm()
storm.t = data[:, 0]
storm.eye_location = data[:, 1:3]

# Plot swath
fig, ax = plt.subplots()
storm.plot(ax, radius=300e3, t_range=[10, 220000])
ax.set_aspect('equal')
ax.set_title("Storm Track - Example 2")
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_xlim((-500e3, 500e3))
ax.set_ylim((-500e3, 500e3))

## Example 3:  ATCF storm file
Fetch the storm file for Hurricane Ike and plot the track, intensity, and radius on a plot with topography taken from the same GeoClaw example.  Here we also restrict the spatial domain to the one used in the example as well as the times used.  Note that the conversion of the radius in km to lat-long is handled automatically in this case.

In [ ]:
scratch_dir = os.path.join(os.environ["CLAW"], 'geoclaw', 'scratch')

# Topography
clawutil.data.get_remote_file("https://depts.washington.edu/clawpack/geoclaw/topo/gulf_caribbean.tt3.tar.bz2")
topo = topotools.Topography(path=os.path.join(scratch_dir, 'gulf_caribbean.tt3'))

# Storm
clawutil.data.get_remote_file("http://ftp.nhc.noaa.gov/atcf/archive/2008/bal092008.dat.gz")
atcf_path = os.path.join(scratch_dir, "bal092008.dat")
with gzip.open(".".join((atcf_path, 'gz')), 'rb') as atcf_file, open(atcf_path, 'w') as atcf_unzipped_file:
    atcf_unzipped_file.write(atcf_file.read().decode('ascii'))
ike = Storm(path=atcf_path, file_format="ATCF")
ike.time_offset = datetime.datetime(2008, 9, 13, 7)

# Plot
fig, ax = plt.subplots()
ax.set_aspect('equal')
topo.plot(ax)
t_range = [ike.time_offset - datetime.timedelta(3), 
           ike.time_offset + datetime.timedelta(1)]
ike.plot(ax, t_range=t_range, track_style={})
ax.set_xlim((-99, -70))
ax.set_ylim((8, 32))